In [1]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader, CSVLoader
import os

In [2]:
# Load documents based on file type
def load_documents(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    if file_extension == '.txt':
        loader = TextLoader(file_path)
    elif file_extension == '.pdf':
        loader = PyPDFLoader(file_path)
    elif file_extension == '.csv':
        loader = CSVLoader(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")
    return loader.load()

In [3]:
# Replace 'your_file_path' with the path to your file (PDF, CSV, or TXT)
file_path = 'data.pdf'  # Change this to your file path
documents = load_documents(file_path)

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

print(f"Split into {len(splits)} chunks.")

Split into 14 chunks.


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import pipeline

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [9]:
query="What are challenges in optimization?"
result = vectorstore.similarity_search(query)
result[0].page_content

'Chapter 5: Optimization Techniques in Deep Learning \nOptimization is the process of adjusting a model’s parameters to minimize a loss function and \nachieve optimal performance. This chapter covers essential optimization techniques, their \nmathematical underpinnings, and practical applications in deep learning. \n \n1. Introduction to Optimization \nOptimization is a core component of deep learning, aiming to find the set of model parameters \n(weights ) that minimize the loss function . \n1.1 Role of Optimization in Neural Networks \n• Adjusts weights and biases to reduce the difference between predicted and actual outputs. \n• Balances computational efficiency with model convergence. \n \n2. Gradient Descent and V ariants \n2.1 Gradient Descent \nThe foundational optimization algorithm for training neural networks. \n• Mathematics: Where: \no : Learning rate. \no : Gradient of the loss function with respect to . \n• Challenges: \no Slow convergence. \no Stuck in local minima or sa

In [ ]:
#Using FAISS to check if it is better than Chroma 

In [11]:
from langchain_community.vectorstores import FAISS
vectorstore2 = FAISS.from_documents(documents=splits, embedding=embeddings)

In [12]:
query="What are challenges in optimization?"
result = vectorstore2.similarity_search(query)
result[0].page_content

'Chapter 5: Optimization Techniques in Deep Learning \nOptimization is the process of adjusting a model’s parameters to minimize a loss function and \nachieve optimal performance. This chapter covers essential optimization techniques, their \nmathematical underpinnings, and practical applications in deep learning. \n \n1. Introduction to Optimization \nOptimization is a core component of deep learning, aiming to find the set of model parameters \n(weights ) that minimize the loss function . \n1.1 Role of Optimization in Neural Networks \n• Adjusts weights and biases to reduce the difference between predicted and actual outputs. \n• Balances computational efficiency with model convergence. \n \n2. Gradient Descent and V ariants \n2.1 Gradient Descent \nThe foundational optimization algorithm for training neural networks. \n• Mathematics: Where: \no : Learning rate. \no : Gradient of the loss function with respect to . \n• Challenges: \no Slow convergence. \no Stuck in local minima or sa

In [ ]:
# FAISS got samilar result as Chroma so can use either

In [13]:
from meta_ai_api import MetaAI

In [14]:
# Initialize MetaAI
ai = MetaAI()

In [15]:
def format_prompt_for_llm(context, question):
    prompt = f"""
    Context:
    {context}

    Question:
    {question}

    Answer:
    """
    return prompt

In [16]:
def get_llm_response(question, vectorstore):
    # Retrieve relevant chunks using similarity search
    relevant_docs = vectorstore.similarity_search(question, k=3)  # Retrieve top 3 relevant chunks
    context = "\n".join([doc.page_content for doc in relevant_docs])
    
    # Format the prompt
    prompt = format_prompt_for_llm(context, question)
    
    # Get response from MetaAI
    response = ai.prompt(message=prompt)
    
    # Extract the message
    message_only = response.get('message', '')
    
    return message_only

In [18]:
# Example usage 1
question = "What are challenges in optimization?"
response = get_llm_response(question, vectorstore)  # Use Chroma vector store
print(f"Question: {question}")
print(f"Response: {response}")

Question: What are challenges in optimization?
Response: The challenges in optimization include:
Slow convergence: Optimization algorithms may take a long time to converge to the optimal solution.
Getting stuck in local minima or saddle points: Optimization algorithms may converge to a local minimum or get stuck in a saddle point, rather than finding the global minimum.



In [21]:
# Example usage 2
question = "What are key features of Scikit-learn?"
response = get_llm_response(question, vectorstore)  # Use Chroma vector store
print(f"Question: {question}")
print(f"Response: {response}")

Question: What are key features of Scikit-learn?
Response: The key features of Scikit-learn include:
Easy-to-use API for fast prototyping: Scikit-learn provides a simple and intuitive API for building and testing machine learning models.
Preprocessing tools for feature scaling, encoding, and selection: Scikit-learn offers various tools for data preprocessing, including feature scaling, encoding categorical variables, and feature selection.
A variety of machine learning algorithms: Scikit-learn supports a wide range of machine learning algorithms, including supervised and unsupervised learning methods.
Model evaluation and validation utilities: Scikit-learn provides tools for evaluating and validating machine learning models, including metrics for performance evaluation and methods for hyperparameter tuning.



In [22]:
# Example usage 3
question = "What are Optimization Techniques in Deep Learning?"
response = get_llm_response(question, vectorstore)  # Use Chroma vector store
print(f"Question: {question}")
print(f"Response: {response}")

Question: What are Optimization Techniques in Deep Learning?
Response: Optimization techniques in deep learning are methods used to adjust a model's parameters to minimize a loss function and achieve optimal performance. The key optimization techniques include:
Gradient Descent (GD): The foundational optimization algorithm for training neural networks.
Stochastic Gradient Descent (SGD): Updates parameters using a single data point or mini-batch.
Mini-Batch Gradient Descent: Processes small batches of data at a time, balancing computational efficiency and stability.
Momentum: Accelerates updates in relevant directions and dampens oscillations.
RMSProp: Scales gradients by a moving average of their squared magnitudes.
Adam (Adaptive Moment Estimation): Combines the benefits of momentum and RMSProp.
These optimization techniques are essential for training deep learning models and achieving optimal performance.

